In [1]:
import pandas as pd
from collections import Counter
import nltk
import re
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

keywordsJson = pd.read_json('../data/depression_synonyms.json', orient='records')
keywords = list(keywordsJson['depression'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bramb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Create vectors

In [2]:
df = pd.read_json('../data/splitData/postdataLinesSplit.json',lines=True)
X_test = df.loc[(df['split'] == 'test')]


def preprocess(post):
    preprocessedWords = []
    lemmatizer = WordNetLemmatizer()
    post = post[0]
    post = post.lower()
    post = re.sub(r'[^\w\s]', '', post)
    words = post.split()
    for word in words:
        lemmatized = lemmatizer.lemmatize(word)
        preprocessedWords.append(lemmatized)
    
    return preprocessedWords

prediction = {'prediction':[]}

for index, row in X_test.iterrows():
    DepressionWordCount = 0
    text = row['text']
    preProcessed = preprocess(text)
    wordCounts = dict(Counter(preProcessed))
    keywordCounts = {}
    keywordCounts['id'] = index
    for word in keywords:
        try:
            DepressionWordCount += wordCounts[word]
        except: 
            continue
    if DepressionWordCount >= 2:
        prediction['prediction'].append('depression')
    else:
        prediction['prediction'].append('normal')
    

vector_df = pd.DataFrame(prediction)
X_test['prediction'] = prediction['prediction']
X_test.to_json("../data/predictionData/ruleBasedPred.json", orient='records', lines=True)

C:\Users\bramb\AppData\Local\Temp\ipykernel_8172\1830165491.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['prediction'] = prediction['prediction']
